# やること

- マルチスレッド化
- tmpファイル生成 -> 削除

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(100)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = (1,1,1,1)

In [2]:
threads = 8
tmp_root = 'tmp'

path_methyl_diff = '../methyl_diff/methyl_diff'

In [3]:
f_pos = '../test/posi.txt'
f_neg = '../test/nega.txt'

In [4]:
df = pd.read_csv(f_pos, sep="\t", header=None)
df.columns=['chr', 'start', 'end', 'P_me', 'P_de', 'P_low', 'P_ratio', 'P_high']

df2= pd.read_csv(f_neg, sep="\t", header=None)
df2.columns=['chr', 'start', 'end', 'N_me', 'N_de', 'N_low', 'N_ratio', 'N_high']

df3= pd.merge(df, df2, on=["chr", "start", "end"])

In [5]:
df3.head()

,chr,start,end,P_me,P_de,P_low,P_ratio,P_high,N_me,N_de,N_low,N_ratio,N_high
0,chrX,170531501,170533500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95
1,chrX,170532001,170534000,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95
2,chrX,170532501,170534500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95
3,chrX,170533001,170535000,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95
4,chrX,170533501,170535500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95


In [6]:
# 別にいらないが。
df_meth = pd.read_csv('../methyl_diff/diff.in', sep=' ', header=None)
df_meth.columns = ['P_me', 'P_de', 'N_me', 'N_de']
df_meth.head()

,P_me,P_de,N_me,N_de
0,0,18,0,9
1,0,3,0,4
2,0,20,0,14
3,0,18,0,19
4,0,20,0,15


In [7]:
index_split = np.array_split(df3.index, threads)

In [8]:
import subprocess
from io import StringIO
import os

os.makedirs(tmp_root, exist_ok=True)

procs=[]
for i in range(threads):
    proc=subprocess.Popen(path_methyl_diff, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    procs.append(proc)


list_out = []
for proc in procs:
    df3.loc[index_split[i], ['P_me', 'P_de', 'N_me', 'N_de']].to_csv(
        tmp_root+'/tmp_meth.{}.txt'.format(i), sep=' ', header=None, index=None)
    data = open(tmp_root+'/tmp_meth.{}.txt'.format(i), 'r').read()
    stdout_data, stderr_data = proc.communicate(data.encode('utf-8'))

    list_out.append(pd.read_csv(StringIO(stdout_data.decode('utf-8')), header=None))
    os.remove(tmp_root+'/tmp_meth.{}.txt'.format(i))
    
if os.listdir(tmp_root) == []:
    os.rmdir(tmp_root)

df3['out'] = list(pd.concat(list_out).reset_index(drop=True)[0])

In [9]:
df3.head()

,chr,start,end,P_me,P_de,P_low,P_ratio,P_high,N_me,N_de,N_low,N_ratio,N_high,out
0,chrX,170531501,170533500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95,0.5
1,chrX,170532001,170534000,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95,0.5
2,chrX,170532501,170534500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95,0.5
3,chrX,170533001,170535000,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95,0.5
4,chrX,170533501,170535500,0,0,0.05,0.5,0.95,0,0,0.05,0.5,0.95,0.5
